In [ ]:
using PyCall
using PyPlot

using Statistics
using DelimitedFiles

using Dates

yf = pyimport("yfinance")
np = pyimport("numpy")
pd = pyimport("pandas")

include("OptimPortfolio.jl")
include("Utils.jl")

rcParams = PyDict(matplotlib["rcParams"])

rcParams["font.size"] = 20
rcParams["axes.labelsize"] = 30

rcParams["xtick.labelsize"] = 16
rcParams["ytick.labelsize"] = 16

rcParams["figure.figsize"] = (8,6)

L2D = PyPlot.matplotlib.lines.Line2D
Ellipse = PyPlot.matplotlib.patches.Ellipse

# Data donwload and basic analysis 

In [ ]:
start = "2013-01-01"
finish = "2017-01-01" 

start_test = "2017-01-01"
finish_test_1  ="2018-01-01"
finish_test_2  ="2019-01-01"
finish_test_3  ="2020-01-01"

assets = ["AAPL", "MSFT", "AMZN",  "TSLA", "GOOGL",  "GOOG", "UNH", "NVDA", "JNJ", "META"] #"^GSPC"

df = yf.download(assets, start, finish, progress=false)

df_test_1 = yf.download(assets, start_test, finish_test_1, progress=false)
df_test_2 = yf.download(assets, start_test, finish_test_2, progress=false)
df_test_3 = yf.download(assets, start_test, finish_test_3, progress=false)

market = yf.download("^GSPC", start, finish, progress=false)

market_test_1 = yf.download("^GSPC", start_test, finish_test_1, progress=false)
market_test_2 = yf.download("^GSPC", start_test, finish_test_2, progress=false)
market_test_3 = yf.download("^GSPC", start_test, finish_test_3, progress=false)

df_log_ret = (df["Adj Close"] / df["Adj Close"].shift(1)).apply(np.log)
df_test_log_ret = (df_test_1["Adj Close"] / df_test_1["Adj Close"].shift(1)).apply(np.log)

market_log_ret = (market["Adj Close"] / market["Adj Close"].shift(1)).apply(np.log)
market_test_log_ret = (market_test_1["Adj Close"] / market_test_1["Adj Close"].shift(1)).apply(np.log);

## Asset analysis

In [ ]:
legend_elements = []

i = 0

for asset in df["Adj Close"].columns
   
    c = "C$(i)"
        
    append!(legend_elements, [L2D([0], [0], color=c, lw=4, label=asset)])
    
    i += 1
    
end

fig, ax = plt.subplot_mosaic("""ABGJ
    CDHK
    EFIL""", figsize=(8*4, 6*3))

# Stock prices
df["Adj Close"].plot(ax=ax["A"], lw=3)

ax["A"].legend(handles=legend_elements, ncol=10, loc="upper center", bbox_to_anchor=(2.3, 1.3), fontsize=24)

ax["A"].set_xlabel("")

df_test_1["Adj Close"].plot(ax=ax["B"], legend=false, lw=3)
ax["B"].set_xlabel("")

df_test_2["Adj Close"].plot(ax=ax["G"], legend=false, lw=3)
ax["G"].set_xlabel("")

df_test_3["Adj Close"].plot(ax=ax["J"], legend=false, lw=3)
ax["J"].set_xlabel("")

#Returns
df["Adj Close"].pct_change().plot(ax=ax["C"], legend=false)

ax["C"].set_xlabel("")

df_test_1["Adj Close"].pct_change().plot(ax=ax["D"], legend=false)
ax["D"].set_xlabel("")

df_test_2["Adj Close"].pct_change().plot(ax=ax["H"], legend=false)
ax["H"].set_xlabel("")

df_test_3["Adj Close"].pct_change().plot(ax=ax["K"], legend=false)
ax["K"].set_xlabel("")

#Histogram of returns
for asset in assets
    df["Adj Close"][asset].pct_change().hist(ax=ax["E"], bins=30, alpha=0.8, density=true, label=asset,
        legend=false)
    #df_log_ret[asset].hist(ax=ax["E"], bins=50, alpha=0.8, label=asset)
end

ax["E"].set_xlim(-0.1, 0.1)

for asset in assets
    df_test_1["Adj Close"][asset].pct_change().hist(ax=ax["F"], bins=20, alpha=0.8, density=true, label=asset,
    legend=false)
    #df_test_1_log_ret[asset].hist(ax=ax["F"], bins=30, alpha=0.8, label=asset)
end
ax["F"].set_xlim(-0.1, 0.1)

for asset in assets
    df_test_2["Adj Close"][asset].pct_change().hist(ax=ax["I"], bins=30, alpha=0.8, density=true, label=asset,
    legend=false)
    #df_test_1_log_ret[asset].hist(ax=ax["F"], bins=30, alpha=0.8, label=asset)
end
ax["I"].set_xlim(-0.1, 0.1)

for asset in assets
    df_test_3["Adj Close"][asset].pct_change().hist(ax=ax["L"], bins=40, alpha=0.8, density=true, label=asset,
    legend=false)
    #df_test_1_log_ret[asset].hist(ax=ax["F"], bins=30, alpha=0.8, label=asset)
end
ax["L"].set_xlim(-0.1, 0.1)

plt.subplots_adjust(wspace=0.25, hspace=0.4)

## Market analysis 

In [ ]:
fig, ax = plt.subplot_mosaic("""ABGJ
    CDHK
    EFIL""", figsize=(8*4, 6*3))

# Stock prices
market["Adj Close"].plot(ax=ax["A"], legend=false, lw=3)

market_test_1["Adj Close"].plot(ax=ax["B"], legend=false, lw=3)
market_test_2["Adj Close"].plot(ax=ax["G"], legend=false, lw=3)
market_test_3["Adj Close"].plot(ax=ax["J"], legend=false, lw=3)

#Returns
market["Adj Close"].pct_change().plot(ax=ax["C"], legend=false)

market_test_1["Adj Close"].pct_change().plot(ax=ax["D"], legend=false)
market_test_2["Adj Close"].pct_change().plot(ax=ax["H"], legend=false)
market_test_3["Adj Close"].pct_change().plot(ax=ax["K"], legend=false)

#Histogram of returns
market["Adj Close"].pct_change().hist(ax=ax["E"], bins=50, alpha=0.8, legend=false)

market_test_1["Adj Close"].pct_change().hist(ax=ax["F"], bins=24, alpha=0.8, legend=false)
market_test_2["Adj Close"].pct_change().hist(ax=ax["I"], bins=30, alpha=0.8, legend=false)
market_test_3["Adj Close"].pct_change().hist(ax=ax["L"], bins=30, alpha=0.8, legend=false)

for item in ax
   
    item[2].set_xlabel("")
    
end

plt.subplots_adjust(wspace=0.25, hspace=0.4)

# Resample data to 1-month frequency

In [ ]:
freq = "1M"

df = df.resample(freq).mean()

df_test_1 = df_test_1.resample(freq).mean()
df_test_2 = df_test_2.resample(freq).mean()
df_test_3 = df_test_3.resample(freq).mean()

market = market.resample(freq).mean()

market_test_1 = market_test_1.resample(freq).mean()
market_test_2 = market_test_2.resample(freq).mean()
market_test_3 = market_test_3.resample(freq).mean()

df_log_ret = (df["Adj Close"] / df["Adj Close"].shift(1)).apply(np.log)
df_test_1_log_ret = (df_test_1["Adj Close"] / df_test_1["Adj Close"].shift(1)).apply(np.log)
df_test_2_log_ret = (df_test_2["Adj Close"] / df_test_2["Adj Close"].shift(1)).apply(np.log)
df_test_3_log_ret = (df_test_3["Adj Close"] / df_test_3["Adj Close"].shift(1)).apply(np.log)

market_log_ret = (market["Adj Close"] / market["Adj Close"].shift(1)).apply(np.log)
market_test_log_ret = (market_test_1["Adj Close"] / market_test_1["Adj Close"].shift(1)).apply(np.log);

# Initialize portfolios 

In [ ]:
returns = df["Adj Close"].pct_change().dropna().values
log_returns = df_log_ret.dropna().values

returns_test_1 = df_test_1["Adj Close"].pct_change().dropna().values
returns_test_2 = df_test_2["Adj Close"].pct_change().dropna().values
returns_test_3 = df_test_3["Adj Close"].pct_change().dropna().values

log_returns_test_1 = df_test_1_log_ret.dropna().values
log_returns_test_2 = df_test_2_log_ret.dropna().values
log_returns_test_3 = df_test_3_log_ret.dropna().values

assets = [item for item in df["Adj Close"].columns]

portfolio = create_portfolio(returns, assets)
portfolio_log = create_portfolio(log_returns, assets)

portfolio_test_1 = create_portfolio(returns_test_1, assets)
portfolio_test_2 = create_portfolio(returns_test_2, assets)
portfolio_test_3 = create_portfolio(returns_test_3, assets)

portfolio_log_test_1 = create_portfolio(log_returns_test_1, assets)
portfolio_log_test_2 = create_portfolio(log_returns_test_2, assets)
portfolio_log_test_3 = create_portfolio(log_returns_test_3, assets);

# Mean-Variance portfolio optimization 

### Fixed returns 

In [ ]:
target_return = 0.03

return_p, risk_p, w_opt = MV_fixed_return(portfolio, target_return; method="DCP")

return_market = market["Adj Close"].pct_change().mean()
risk_market = sqrt(market["Adj Close"].pct_change().var())

println("Expected Returns Portfolio: ", return_p)
println("Risk Portfolio: ", risk_p)
#println("Weights:", w_opt)

println("\nExpected Return Market: ", return_market)
println("Risk Market: ", risk_market)

println("\nPortfolio-market return ratio: ", return_p/return_market)
println("Portfolio-market risk ratio: ", risk_p/risk_market)

### Fixed risk 

In [ ]:
target_risk = 0.035

return_p, risk_p, w_opt = MV_fixed_risk(portfolio, target_risk; method="DCP")

return_market = market["Adj Close"].pct_change().mean()
risk_market = sqrt(market["Adj Close"].pct_change().var())

println("Expected Returns Portfolio: ", return_p)
println("Risk Portfolio: ", risk_p)
#println("Weights:", w_opt)

println("\nExpected Return Market: ", return_market)
println("Risk Market: ", risk_market)

println("\nPortfolio-market return ratio: ", return_p/return_market)
println("Portfolio-market risk ratio: ", risk_p/risk_market)

# Efficient frontier 

In [ ]:
RetRisk, weights = MV_efficient_frontier(portfolio, 100);

RetRisk_log, weights_log = MV_efficient_frontier(portfolio_log, 100);

risks = sqrt.(diag(portfolio.Σ))
risks_log = sqrt.(diag(portfolio_log.Σ))

##### FIGURE ######

legend_elements = []

i = 0

for asset in portfolio.assets
   
    c = "C$(i)"
        
    append!(legend_elements, [L2D([0], [0], color=c, lw=4, label=asset)])
    
    i += 1
    
end

append!(legend_elements, [L2D([0], [0], ls="", marker="d", color="k", markersize=12, label="S&P500")])
append!(legend_elements, [L2D([0], [0], ls="--", lw=3, color="k", label="Efficient frontier")])


plt.figure(figsize=(8*2, 6*2))

plt.subplot(2, 2, 1)

for i in 1:length(portfolio.μ)

    plt.scatter(risks[i], portfolio.μ[i], s=100)
    
end

plt.scatter(risk_market, return_market, s=100, color="k", marker="d", label="S&P500")

plt.plot(RetRisk[:, 2], RetRisk[:, 1], color="k", lw=3, ls="--", label="Efficient frontier")

plt.ylabel("Expected return", fontsize=20, labelpad=15)
plt.xlabel("Risk", fontsize=20, labelpad=15)

plt.legend(handles=legend_elements, ncol=6, loc="upper center", bbox_to_anchor=(1.05, 1.4))

plt.subplot(2, 2, 2)

for i in 1:length(portfolio.μ)

    plt.scatter(risks_log[i], portfolio_log.μ[i], s=100)
    
end

plt.scatter(risk_market, return_market, s=100, color="k", marker="d", label="S&P500")

plt.plot(RetRisk_log[:, 2], RetRisk_log[:, 1], color="k", lw=3, ls="--", label="Efficient frontier")

plt.ylabel("Expected log return", fontsize=20, labelpad=15)
plt.xlabel("Risk", fontsize=20, labelpad=15)

#plt.legend(ncol=2)

#
plt.subplot(2,2,3)

N = Int(10^6)

rets = zeros(N)
risks = zeros(N)
sharpe_ratio = zeros(N)

for i in 1 : N

    w = rand(length(portfolio.μ))

    w = w / sum(w)
   
    return_p = dot(w, portfolio.μ)
    risk_p = sqrt(transpose(w)*portfolio.Σ*w)
    
    rets[i] = return_p
    risks[i] = risk_p
    
    sharpe_ratio[i] = return_p / risk_p
    
end

plt.plot(RetRisk[:, 2], RetRisk[:, 1], color="k", lw=3, ls="--", label="Efficient frontier")

plt.scatter(risks, rets, c=sharpe_ratio, s=8)

plt.ylabel("Expected return", fontsize=20, labelpad=15)
plt.xlabel("Risk", fontsize=20, labelpad=15)

plt.colorbar()

#
plt.subplot(2,2,4)

N = Int(10^6)

rets = zeros(N)
risks = zeros(N)
sharpe_ratio = zeros(N)

for i in 1 : N

    w = rand(length(portfolio_log.μ))

    w = w / sum(w)
   
    return_p = dot(w, portfolio_log.μ)
    risk_p = sqrt(transpose(w)*portfolio_log.Σ*w)
    
    rets[i] = return_p
    risks[i] = risk_p
    
    sharpe_ratio[i] = return_p / risk_p
    
end

plt.plot(RetRisk_log[:, 2], RetRisk_log[:, 1], color="k", lw=3, ls="--", label="Efficient frontier")

plt.scatter(risks, rets, c=sharpe_ratio, s=8)

plt.ylabel("Expected log return", fontsize=20, labelpad=15)
plt.xlabel("Risk", fontsize=20, labelpad=15)

plt.colorbar()

plt.subplots_adjust(wspace=0.3, hspace=0.3)

# Portfolio testing 

Up to this moment we have optimized the weights of our portfolio for any desired level of returns and risk. However, this is done in a training (in-sample) set, but in the future things could change and our optimal weights could'nt be optimal any more. Here we check how expected returns and expected risk for the optimized weights could vary in the future by applying our optimal weights to a test (out of sample) set. We will show 2 results:

- An example of the out-of-sample performance of the in-sample optimized portfolio yielding a fixed return of 0.025

- The Yield Curve of all in-sample optimal portfolios (the ones on the efficient frontier) in the out-of-sample set.

In [ ]:
#EXAMPLE

#Test in "future" data
#df_test_returns = df_test["Adj Close"].pct_change().dropna().values

target_return = 0.025

#Optimize in past data
ret_opt, risk_opt, w_opt = MV_fixed_return(portfolio, target_return)

returns_p_opt_test_1 = returns_test_1 * w_opt
returns_p_opt_test_2 = returns_test_2 * w_opt
returns_p_opt_test_3 = returns_test_3 * w_opt
    
risk = round(sqrt(var(returns_test_1)), digits=3)

dates_1 = [df_test_1["Adj Close"].index[i] for i in 2: length(df_test_1)]
dates_2 = [df_test_2["Adj Close"].index[i] for i in 2: length(df_test_2)]
dates_3 = [df_test_3["Adj Close"].index[i] for i in 2: length(df_test_3)]

#mean_return = round(mean(returns_test), digits=4)
#return_market = round(market_test["Adj Close"].pct_change().mean(), digits=4)
#risk_market = round(sqrt(market_test["Adj Close"].pct_change().var()), digits=3);

target_returns = 0.021 : 0.002 : 0.048

final_risks_1 = []
final_returns_1 = []

final_risks_2 = []
final_returns_2 = []

final_risks_3 = []
final_returns_3 = []

expected_risks = []
expected_returns = []

for target_return in target_returns

    expected_return, expected_risk, w_opt = MV_fixed_return(portfolio, target_return)

    returns_1 = returns_test_1 * w_opt
    returns_2 = returns_test_2 * w_opt
    returns_3 = returns_test_3 * w_opt

    actual_risk_1 = round(sqrt(var(returns_1)), digits=6)
    actual_mean_return_1 = round(mean(returns_1), digits=6)
    
    actual_risk_2 = round(sqrt(var(returns_2)), digits=6)
    actual_mean_return_2 = round(mean(returns_2), digits=6)
    
    actual_risk_3 = round(sqrt(var(returns_3)), digits=6)
    actual_mean_return_3 = round(mean(returns_3), digits=6)
    
    append!(expected_risks, expected_risk)
    append!(expected_returns, expected_return)
    
    append!(final_risks_1, actual_risk_1)
    append!(final_returns_1, actual_mean_return_1)
    
    append!(final_risks_2, actual_risk_2)
    append!(final_returns_2, actual_mean_return_2)
    
    append!(final_risks_3, actual_risk_3)
    append!(final_returns_3, actual_mean_return_3)
    
end

RetRisk_test_1, weights_1 = MV_efficient_frontier(portfolio_test_1, 50)
RetRisk_test_2, weights_2 = MV_efficient_frontier(portfolio_test_2, 25)
RetRisk_test_3, weights_3 = MV_efficient_frontier(portfolio_test_3, 25);

In [ ]:
fig, ax = plt.subplot_mosaic("AB", figsize=(8*2,6))

#ax["A"].plot(dates_1, mean(returns_test_1, dims=2), marker="o", ms=8, label="Portfolio simulated returns")
#ax["A"].plot(dates_2, mean(returns_test_2, dims=2), marker="o", ms=8, label="Portfolio simulated returns")
ax["A"].plot(dates_3, mean(returns_test_3, dims=2), marker="o", ms=8, label="Portfolio simulated returns")

ax["A"].plot(dates_1, [-0.06 for item in dates_1], color="g", lw=3, label="Test 1")
ax["A"].plot(dates_2, [-0.07 for item in dates_2], lw=3, color="orange", label="Test 1")
ax["A"].plot(dates_3, [-0.08 for item in dates_3], lw=3, color="r", label="Test 3")

#ax["A"].axhline(mean_return, color="k", lw=3, label="Portfolio: μ=$mean_return,  σ=$risk")
#ax["A"].axhline(return_market, color="k", lw=3, ls="--", label="Market: μ=$return_market,  σ=$risk_market")

ax["A"].tick_params("x", rotation=45)

#ax["A"].set_ylim(0, 0.08)

ax["A"].legend(loc="upper center",ncol=2, bbox_to_anchor=(0.5, 1.3))

#ax["B"].scatter(risk_market, return_market, color="k", s=100, marker="d", label="S&P500")

ax["B"].plot(expected_risks, expected_returns, color="k", lw=5)

ax["B"].plot(final_risks_1, final_returns_1, marker="o", c="C0", ms=8)
ax["B"].plot(final_risks_2, final_returns_2, marker="^", c="C0", ms=8)
ax["B"].plot(final_risks_3, final_returns_3, marker="s", c="C0", ms=8)

ax["B"].plot(RetRisk_test_1[:, 2], RetRisk_test_1[:, 1], color="C1", ls="-", marker="o", ms=8, 
    label="Test 1 Efficient Frontier")
ax["B"].plot(RetRisk_test_2[:, 2], RetRisk_test_2[:, 1], color="C1", ls="-", marker="^", ms=8, 
    label="Test 2 Efficient Frontier")
ax["B"].plot(RetRisk_test_3[:, 2], RetRisk_test_3[:, 1], color="C1", ls="-", marker="s", ms=8, 
    label="Test 3 Efficient Frontier")

ax["B"].set_ylabel("Efficient return", fontsize=20, labelpad=15)
ax["B"].set_xlabel("Risk", fontsize=20, labelpad=15)

legend_elements = [L2D([0], [0], lw=5, color="k", label="In-sample efficient frontier"),
                   L2D([0], [0], lw=5, color="C1", label="Test efficient frontier"),
                   L2D([0], [0], lw=5, color="C0", label="Test portfolio performance"),
                   L2D([0], [0], ls="", marker="o", color="k", ms=10, label="Test 1"),
                   L2D([0], [0], ls="", marker="^", color="k", ms=10, label="Test 2"),
                   L2D([0], [0], ls="", marker="s", color="k", ms=10, label="Test 3")]

ax["B"].legend(handles=legend_elements, loc="upper center", ncol=2, bbox_to_anchor=(0.5, 1.35))

plt.subplots_adjust(wspace=0.3)

- The in-sample optimized portfolios are always suboptimal in the testing period (which could be expected). However, we observe that if the testing period is near the training period (so if it shprt enough) the in-sample optimized portfolios are a good approximation to the test efficient frontier.

- It is interesting to observe that, for the near-future case, the in-sample optimized portfolios indeed outperform the expected results from the in-sample efficient frontier

- We also note that the efficient frontiers of the 2nd and rd testing periods are quite different from that of the 1st period, showing that the optimization problem is harder. This is probably because the high volatility in those periods.

# Portfolio Evolutionary Optimization 

## Sanity check with MV model 

### Fixed return optimization 

In [ ]:
target_return = 0.03

return_p_DCP, risk_p_DCP, w_opt_DCP = MV_fixed_return(portfolio, target_return; method="DCP")
return_p_EO, risk_p_EO, w_opt_EO = MV_fixed_return(portfolio, target_return; method="EO")

println("Expected Returns Portfolio (DCP): ", return_p_DCP)
println("Expected Returns Portfolio (EO): ", return_p_EO)

println("\nRisk Portfolio (DCP): ", risk_p_DCP)
println("Risk Portfolio (EO): ", risk_p_EO)

### Fixed risk optimization 

In [ ]:
target_risk = 0.035

return_p_DCP, risk_p_DCP, w_opt_DCP = MV_fixed_risk(portfolio, target_risk; method="DCP")
return_p_EO, risk_p_EO, w_opt_EO = MV_fixed_risk(portfolio, target_risk; method="EO")

println("Expected Returns Portfolio (DCP): ", return_p_DCP)
println("Expected Returns Portfolio (EO): ", return_p_EO)

println("\nRisk Portfolio (DCP): ", risk_p_DCP)
println("Risk Portfolio (EO): ", risk_p_EO)

### Efficient Frontier 

In [ ]:
#Efficient frontier using Convex Optimization
@time RetRisk_DCP, weights_DCP = MV_efficient_frontier(portfolio, 50; method="DCP")

#Efficient frontier using Evolutionary Optimization
@time RetRisk_EO, weights_EO = MV_efficient_frontier(portfolio, 30; method="EO");

plt.plot(RetRisk_DCP[:, 2], RetRisk_DCP[:, 1], color="g", lw=3, zorder=1, label="DCP")
plt.scatter(RetRisk_EO[:, 2], RetRisk_EO[:, 1], color="r", label="EO")

plt.legend()

# Higher-Moment Portfolio Optimization 

Now the optimization problem is non-convex, so DCP can not be used. We will optimize the portfolios under MVSK model using Evolutionary Optimization.

## High-dimensional efficient frontier 

This optimization problem is highly computationally demanding, so we run the simulations in a cluster. Here we load the data and show some results.

In [ ]:
#Generate random portfolios anc compute M-V-S-K
N = Int(10^4)

rets = zeros(N)
risks = zeros(N)
skews = zeros(N)
kurts = zeros(N)

sharpe_ratio = zeros(N)

@time for i in 1 : N

    w = rand(length(portfolio.μ))

    w = w / sum(w)
   
    return_p = dot(w, portfolio.μ)
    risk_p = sqrt(transpose(w)*portfolio.Σ*w)
    skew_p = skewness_portfolio(portfolio, w)
    kurt_p = kurtosis_portfolio(portfolio, w)
    
    rets[i] = return_p
    risks[i] = risk_p
    skews[i] = skew_p
    kurts[i] = kurt_p
    
    sharpe_ratio[i] = return_p / risk_p
    
end

RetRisk_DCP, weights_DCP = MV_efficient_frontier(portfolio, 50; method="DCP")

#Load optimized portfolios
filenames = readdir("HD_efficient_frontier")

results = zeros((length(filenames), 4))
λs = zeros((length(filenames), 4))
weights = zeros((length(filenames), 10))

i = 1

for filename in filenames
   
    data, header = readdlm(string("HD_efficient_frontier/", filename), header=true)
   
    results[i, :] = data[1:4]
    λs[i, :] = data[5:8]
    weights[i, :] = data[9:end]
    
    i += 1
    
end

results_MV = sort(results[(λs[:, 3] .== 0.0) .& (λs[:, 4] .== 0.0), :], dims=1)
results_MK = sort(results[(λs[:, 2] .== 0.0) .& (λs[:, 3] .== 0.0), :], dims=1)
results_SK = sort(results[(λs[:, 1] .== 0.0) .& (λs[:, 2] .== 0.0), :], dims=1);

In [ ]:
fig, ax = plt.subplot_mosaic("""AB
    CD""", figsize=(8*2, 6*2))

ax["A"].scatter(kurts, skews, c=rets, cmap="jet")
ax["A"].plot(results_SK[:, 4], results_SK[:, 3], color="k", lw=3, ls="--")

ax["A"].set_xlim(0.1, 1.2)
ax["A"].set_ylim(-0.15, 0.3)

ax["B"].scatter(kurts, skews, c=risks, cmap="jet")
ax["B"].plot(results_SK[:, 4], results_SK[:, 3], color="k", lw=3, ls="--")

ax["B"].set_xlim(0.1, 1.2)
ax["B"].set_ylim(-0.15, 0.3)

ax["C"].scatter(risks, rets, c=skews, cmap="jet")
ax["C"].plot(results_MV[:, 2], results_MV[:, 1], color="k", lw=3, ls="--", label="Efficient frontier")

ax["D"].scatter(risks, rets, c=kurts, cmap="jet")
ax["D"].plot(results_MV[:, 2], results_MV[:, 1], color="k", lw=3, ls="--", label="Efficient frontier")

## In-Sample Portfolio Optimization

In [ ]:
fig, ax = plt.subplot_mosaic("""AB
CD""", figsize=(8*2,6*2))

cn1 = ax["A"].scatter(results[:, 2], results[:, 1], c=results[:, 3], s=60, cmap="Set2")

ax["A"].plot(results_MV[:, 2], results_MV[:, 1], lw=3, color="k", ls="-", label="M-V")
ax["A"].plot(results_MK[:, 2], results_MK[:, 1], lw=3, color="k", ls="--", label="M-K")
ax["A"].plot(results_SK[:, 2], results_SK[:, 1], lw=3, color="k", ls=":", label="S-K")

ax["A"].legend(loc="upper center", ncol=3, bbox_to_anchor=(1.25, 1.3))

cbar1 = fig.colorbar(cn1, ax=ax["A"])
cbar1.ax.set_title("Skewness", pad=10, fontsize=16)

ax["A"].set_xlabel("Risk")
ax["A"].set_ylabel("Return")

cn2 = ax["B"].scatter(results[:, 2], results[:, 1], c=results[:, 4], s=60, cmap="Set2")

cbar2 = fig.colorbar(cn2, ax=ax["B"])
cbar2.ax.set_title("Kurtosis", pad=10, fontsize=16)

ax["B"].plot(results_MV[:, 2], results_MV[:, 1], lw=3, color="k", ls="-", label="M-V")
ax["B"].plot(results_MK[:, 2], results_MK[:, 1], lw=3, color="k", ls="--", label="M-K")
ax["B"].plot(results_SK[:, 2], results_SK[:, 1], lw=3, color="k", ls=":", label="S-K")

ax["B"].set_xlabel("Risk")
ax["B"].set_ylabel("Return")

cn3 = ax["C"].scatter(results[:, 4], results[:, 3], c=results[:, 1], s=60, cmap="Set2")

cbar3 = fig.colorbar(cn3, ax=ax["C"])
cbar3.ax.set_title("Returns", pad=10, fontsize=16)

ax["C"].plot(results_MV[:, 4], results_MV[:, 3], lw=3, color="k", ls="-", label="M-V")
ax["C"].plot(results_MK[:, 4], results_MK[:, 3], lw=3, color="k", ls="--", label="M-K")
ax["C"].plot(results_SK[:, 4], results_SK[:, 3], lw=3, color="k", ls=":", label="S-K")

axinsC = ax["C"].inset_axes([0.5, 0.1, 0.4, 0.4])

axinsC.scatter(results[:, 4], results[:, 3], c=results[:, 2], s=60, cmap="Set2")
axinsC.plot(results_MV[:, 4], results_MV[:, 3], lw=3, color="k", ls="-", label="M-V")
axinsC.plot(results_MK[:, 4], results_MK[:, 3], lw=3, color="k", ls="--", label="M-K")
axinsC.plot(results_SK[:, 4], results_SK[:, 3], lw=3, color="k", ls=":", label="S-K")

axinsC.set_xlim(0.1, 1.2)
axinsC.set_ylim(-0.1, 0.25)

axinsC.set_xticks([])
axinsC.set_yticks([])

ax["C"].indicate_inset_zoom(axinsC, edgecolor="black")

ax["C"].set_xlabel("Kurtosis")
ax["C"].set_ylabel("Skewness")

cn4 = ax["D"].scatter(results[:, 4], results[:, 3], c=results[:, 2], s=60, cmap="Set2")

cbar4 = fig.colorbar(cn4, ax=ax["D"])
cbar4.ax.set_title("Risk", pad=10, fontsize=16)

ax["D"].plot(results_MV[:, 4], results_MV[:, 3], lw=3, color="k", ls="-", label="M-V")
ax["D"].plot(results_MK[:, 4], results_MK[:, 3], lw=3, color="k", ls="--", label="M-K")
ax["D"].plot(results_SK[:, 4], results_SK[:, 3], lw=3, color="k", ls=":", label="S-K")

axinsD = ax["D"].inset_axes([0.5, 0.1, 0.4, 0.4])

axinsD.scatter(results[:, 4], results[:, 3], c=results[:, 2], s=60, cmap="Set2")
axinsD.plot(results_MV[:, 4], results_MV[:, 3], lw=3, color="k", ls="-", label="M-V")
axinsD.plot(results_MK[:, 4], results_MK[:, 3], lw=3, color="k", ls="--", label="M-K")
axinsD.plot(results_SK[:, 4], results_SK[:, 3], lw=3, color="k", ls=":", label="S-K")

axinsD.set_xlim(0.1, 1.2)
axinsD.set_ylim(-0.1, 0.25)

axinsD.set_xticks([])
axinsD.set_yticks([])

ax["D"].indicate_inset_zoom(axinsD, edgecolor="black")

ax["D"].set_xlabel("Kurtosis")
ax["D"].set_ylabel("Skewness")

plt.subplots_adjust(wspace=0.25, hspace=0.25)

# Out of sample Higher Moment Portfolio Optimization 

In [ ]:
#Be careful, StatsBase.jl kurtosis() function computes the standarized kurtosis!

N_portfolios = size(weights)[1]

results_p_opt_1 = zeros((N_portfolios, 4))
results_p_opt_2 = zeros((N_portfolios, 4))
results_p_opt_3 = zeros((N_portfolios, 4))

for i in 1 : N_portfolios
    
    res_i_1 = returns_test_1 * weights[i, :]
    res_i_2 = returns_test_2 * weights[i, :]
    res_i_3 = returns_test_3 * weights[i, :]
    
    results_p_opt_1[i, :] = [mean(res_i_1), sqrt(var(res_i_1)), skewness(res_i_1), kurtosis(res_i_1)+3]
    results_p_opt_2[i, :] = [mean(res_i_2), sqrt(var(res_i_2)), skewness(res_i_2), kurtosis(res_i_2)+3]
    results_p_opt_3[i, :] = [mean(res_i_3), sqrt(var(res_i_3)), skewness(res_i_3), kurtosis(res_i_3)+3]
        
end

results_MV_OOS_1 = sort(results_p_opt_1[(λs[:, 3] .== 0.0) .& (λs[:, 4] .== 0.0), :], dims=1)
results_MK_OOS_1 = sort(results_p_opt_1[(λs[:, 2] .== 0.0) .& (λs[:, 3] .== 0.0), :], dims=1)
results_SK_OOS_1 = sort(results_p_opt_1[(λs[:, 1] .== 0.0) .& (λs[:, 2] .== 0.0), :], dims=1)

results_MV_OOS_2 = sort(results_p_opt_2[(λs[:, 3] .== 0.0) .& (λs[:, 4] .== 0.0), :], dims=1)
results_MK_OOS_2 = sort(results_p_opt_2[(λs[:, 2] .== 0.0) .& (λs[:, 3] .== 0.0), :], dims=1)
results_SK_OOS_2 = sort(results_p_opt_2[(λs[:, 1] .== 0.0) .& (λs[:, 2] .== 0.0), :], dims=1)

results_MV_OOS_3 = sort(results_p_opt_3[(λs[:, 3] .== 0.0) .& (λs[:, 4] .== 0.0), :], dims=1)
results_MK_OOS_3 = sort(results_p_opt_3[(λs[:, 2] .== 0.0) .& (λs[:, 3] .== 0.0), :], dims=1)
results_SK_OOS_3 = sort(results_p_opt_3[(λs[:, 1] .== 0.0) .& (λs[:, 2] .== 0.0), :], dims=1);

In [ ]:
#Load optimized portfolios in the Out of sample period
filenames_OOS_1 = readdir("HD_efficient_frontier_test_1")
filenames_OOS_2 = readdir("HD_efficient_frontier_test_2")
filenames_OOS_3 = readdir("HD_efficient_frontier_test_3")

results_OOS_1 = zeros((length(filenames), 4))
results_OOS_2 = zeros((length(filenames), 4))
results_OOS_3 = zeros((length(filenames), 4))

λs_test_1 = zeros((length(filenames_OOS_1), 4))
λs_test_2 = zeros((length(filenames_OOS_2), 4))
λs_test_3 = zeros((length(filenames_OOS_3), 4))

i = 1

for filename in filenames_OOS_1
   
    data, header = readdlm(string("HD_efficient_frontier_test_1/", filename), header=true)
   
    results_OOS_1[i, :] = data[1:4]
    
    λs_test_1[i, :] = data[5:8]
    
    i += 1
    
end

i = 1

for filename in filenames_OOS_1
   
    data, header = readdlm(string("HD_efficient_frontier_test_2/", filename), header=true)
   
    results_OOS_2[i, :] = data[1:4]
    
    λs_test_2[i, :] = data[5:8]
    
    i += 1
    
end

i = 1

for filename in filenames_OOS_1
   
    data, header = readdlm(string("HD_efficient_frontier_test_3/", filename), header=true)
   
    results_OOS_3[i, :] = data[1:4]
    
    λs_test_3[i, :] = data[5:8]
    
    i += 1
    
end

results_OOS_MV_opt_1 = sort(results_OOS_1[(λs_test_1[:, 3] .== 0.0) .& (λs_test_1[:, 4] .== 0.0), :], dims=1)
results_OOS_MK_opt_1 = sort(results_OOS_1[(λs_test_1[:, 2] .== 0.0) .& (λs_test_1[:, 3] .== 0.0), :], dims=1)
results_OOS_SK_opt_1 = sort(results_OOS_1[(λs_test_1[:, 1] .== 0.0) .& (λs_test_1[:, 2] .== 0.0), :], dims=1)

results_OOS_MV_opt_2 = sort(results_OOS_2[(λs_test_2[:, 3] .== 0.0) .& (λs_test_2[:, 4] .== 0.0), :], dims=1)
results_OOS_MK_opt_2 = sort(results_OOS_2[(λs_test_2[:, 2] .== 0.0) .& (λs_test_2[:, 3] .== 0.0), :], dims=1)
results_OOS_SK_opt_2 = sort(results_OOS_2[(λs_test_2[:, 1] .== 0.0) .& (λs_test_2[:, 2] .== 0.0), :], dims=1)

results_OOS_MV_opt_3 = sort(results_OOS_3[(λs_test_3[:, 3] .== 0.0) .& (λs_test_3[:, 4] .== 0.0), :], dims=1)
results_OOS_MK_opt_3 = sort(results_OOS_3[(λs_test_3[:, 2] .== 0.0) .& (λs_test_3[:, 3] .== 0.0), :], dims=1)
results_OOS_SK_opt_3 = sort(results_OOS_3[(λs_test_3[:, 1] .== 0.0) .& (λs_test_3[:, 2] .== 0.0), :], dims=1);

In [ ]:
## Sharpe ratio
Sharpe_Ratio_p_OOS_1 = results_p_opt_1[:, 1] ./ results_p_opt_1[:, 2]

λ_vals = 0.0 : 0.05 : 1.0

mean_SR_λ = zeros((length(λ_vals), 4))

k = 0

for i in λ_vals

    k += 1
    
    mean_SR_λ[k, 1] = mean(Sharpe_Ratio_p_OOS_1[λs_test_1[:, 1] .== i])
    mean_SR_λ[k, 2] = mean(Sharpe_Ratio_p_OOS_1[λs_test_1[:, 2] .== i])
    mean_SR_λ[k, 3] = mean(Sharpe_Ratio_p_OOS_1[λs_test_1[:, 3] .== i])
    mean_SR_λ[k, 4] = mean(Sharpe_Ratio_p_OOS_1[λs_test_1[:, 4] .== i])
    
end

##

idxs = 1 : length(Sharpe_Ratio_p_OOS_1)

SR_MV = Sharpe_Ratio_p_OOS_1[(λs_test_1[:, 3] .== 0.0) .& (λs_test_1[:, 4] .== 0.0)]
idxs_MV = idxs[(λs_test_1[:, 3] .== 0.0) .& (λs_test_1[:, 4] .== 0.0)]

SR_λ4_05 = Sharpe_Ratio_p_OOS_1[(λs_test_1[:, 4] .== 0.5)]
idxs_λ4_05 = idxs[(λs_test_1[:, 4] .== 0.5)]

SR_λ3_02 = Sharpe_Ratio_p_OOS_1[(λs_test_1[:, 3] .== 0.1)]
idxs_λ3_02 = idxs[(λs_test_1[:, 3] .== 0.1)];

max_SR = maximum(results_OOS_MV_opt_1[:, 1] ./ results_OOS_MV_opt_1[:, 2]);

In [ ]:
legend_elements = [L2D([0], [0], lw=5, color="k", label="In-sample efficient frontier"),
                   L2D([0], [0], lw=5, color="C1", label="Test efficient frontier"),
                   L2D([0], [0], lw=5, color="C0", label="Test portfolio performance"),
                   L2D([0], [0], ls="", marker="o", color="k", ms=10, label="Test 1"),
                   L2D([0], [0], ls="", marker="^", color="k", ms=10, label="Test 2"),
                   L2D([0], [0], ls="", marker="s", color="k", ms=10, label="Test 3")]

fig, ax = plt.subplot_mosaic("""AB
CD""", figsize=(8*2,6*2))

ax["A"].plot(results_MV[:, 2], results_MV[:, 1], lw=5, color="k", label="M-V")

ax["A"].plot(results_OOS_MV_opt_1[:, 2], results_OOS_MV_opt_1[:, 1], color="C1", marker="o", ms=8)
ax["A"].plot(results_OOS_MV_opt_2[:, 2], results_OOS_MV_opt_2[:, 1], color="C1", marker="^", ms=8)
ax["A"].plot(results_OOS_MV_opt_3[:, 2], results_OOS_MV_opt_3[:, 1], color="C1", marker="s", ms=8)

ax["A"].plot(results_MV_OOS_1[:, 2], results_MV_OOS_1[:, 1], color="C0", ls="-", marker="o", ms=8)
ax["A"].plot(results_MV_OOS_2[:, 2], results_MV_OOS_2[:, 1], color="C0", ls="-", marker="^", ms=8)
ax["A"].plot(results_MV_OOS_3[:, 2], results_MV_OOS_3[:, 1], color="C0", ls="-", marker="s", ms=8)

ax["A"].set_title("M-V")
ax["A"].set_ylabel("")

ax["A"].legend(handles=legend_elements, loc="upper center", ncol=3, bbox_to_anchor=(1.05, 1.4))

ax["B"].plot(results_SK[:, 4], results_SK[:, 3], lw=5, color="k", label="S-K")

ax["B"].plot(results_OOS_SK_opt_1[:, 4], results_OOS_SK_opt_1[:, 3], color="C1", marker="o", ms=8)
ax["B"].plot(results_OOS_SK_opt_2[:, 4], results_OOS_SK_opt_2[:, 3], color="C1", marker="^", ms=8)
ax["B"].plot(results_OOS_SK_opt_3[:, 4], results_OOS_SK_opt_3[:, 3], color="C1", marker="s", ms=8)

ax["B"].plot(results_SK_OOS_1[:, 4], results_SK_OOS_1[:, 3], color="C0", ls="-", marker="o", ms=8)
ax["B"].plot(results_SK_OOS_2[:, 4], results_SK_OOS_2[:, 3], color="C0", ls="-", marker="^", ms=8)
ax["B"].plot(results_SK_OOS_3[:, 4], results_SK_OOS_3[:, 3], color="C0", ls="-", marker="s", ms=8)

ax["B"].set_title("S-K")

ax["C"].plot(λ_vals, mean_SR_λ[:, 1], lw=3, label=L"$\lambda_1$")
ax["C"].plot(λ_vals, mean_SR_λ[:, 2], lw=3, label=L"$\lambda_2$")
ax["C"].plot(λ_vals, mean_SR_λ[:, 3], lw=3, label=L"$\lambda_3$")
ax["C"].plot(λ_vals, mean_SR_λ[:, 4], lw=3, label=L"$\lambda_4$")

ax["C"].set_ylabel("Sharpe Ratio")
ax["C"].set_xlabel(L"$\lambda$ value")

ax["C"].legend(ncol=4, loc="upper center", bbox_to_anchor=(0.5, 1.2))

ax["D"].scatter(idxs, Sharpe_Ratio_p_OOS_1, alpha=0.1, color="k")

ax["D"].scatter(idxs_MV, SR_MV, s=50, label=L"$\lambda_3=\lambda_4=0$")

ax["D"].scatter(idxs_λ4_05, SR_λ4_05, s=50, label=L"$\lambda_4=0.5$")
ax["D"].scatter(idxs_λ3_02, SR_λ3_02, s=50, label=L"$\lambda_3=0.1$")

ax["D"].axhline([max_SR], ls="--", color="k")
ax["D"].text(0, 3.4, "Max Sharpe Ratio from test MV efficient frontier", fontsize=14)

ax["D"].set_ylim(0, 3.8)

ax["D"].set_ylabel("Sharpe Ratio")

plt.legend(ncol=3, loc="upper center", bbox_to_anchor=(0.5, 1.2))

plt.subplots_adjust(hspace=0.4, wspace=0.3)

- We observe that in-sample optimized portfolios are suboptimal in the testing period (expected).

- However, we onbserve that these can achieve better performance in the testing period if using MVSK model than MV model (sharpe ratio).

# Dynamic Portfolio Optimization 

In [ ]:
#=
using Distributed

addprocs(4)
    
@everywhere using SharedArrays
@everywhere using PyCall
@everywhere using Statistics
@everywhere using DelimitedFiles
@everywhere using Dates

@everywhere yf = pyimport("yfinance")
@everywhere np = pyimport("numpy")
@everywhere pd = pyimport("pandas")

@everywhere include("OptimPortfolio.jl")
@everywhere include("Utils.jl")

@everywhere 
=#

function dynamic_optimization(assets, start_date, end_date, freq, investment_horizon, λ_MV, 
        λ_MVSK; w_upper=1)
    
    if freq == "1M"
                
        new_date = start_date - Month(investment_horizon)
        
        periods = length(start_date:Month(1):end_date)
        
    elseif freq == "1D"
        
        new_date = start_date - Day(investment_horizon)
        
        periods = Dates.days(start_date - start_date)
                
    elseif freq == "1Y"
        
        new_date = start_date - Year(investment_horizon)
        
        periods = length(start_date:Year(1):end_date)
                
    else
       
        println("Frequency not supported")
        
    end
        
    df_dynamic = yf.download(assets, new_date, end_date, progress=false).resample(freq).mean()

    realized_returns = zeros((periods-1, 5)) #MV-dynamic, MV-Static, MVSK-dynamic, MVSK-static, EW 

    portfolio_value = ones((periods, 5)) #MV-dynamic, MV-Static, MVSK-dynamic, MVSK-static, EW 
    
    weights = zeros((periods-1, length(assets), 2))
                
    returns = df_dynamic["Adj Close"].pct_change().dropna().values

    returns_i = returns[1 : investment_horizon, :]

    portfolio_i = create_portfolio(returns_i, assets);

    return_MV, risk_MV, w_opt_MV = MV(portfolio_i, λ_MV; method="DCP", w_upper=w_upper)
    w_opt_MV_static = copy(w_opt_MV)
    
    #=
    results, w_opt_MVSK = MVSK(portfolio_i, λ_MVSK; w_upper=w_upper)
    w_opt_MVSK_static = copy(w_opt_MVSK)
    =#
    
    i = 0
    
    while start_date < end_date
        
        i += 1
        
        #Check performance        
        returns_delta = returns[investment_horizon + i - 1 : investment_horizon + i - 1, :]

        returns_p_MV = (returns_delta * w_opt_MV)[1]
        returns_p_MV_static = (returns_delta * w_opt_MV_static)[1]
        #returns_p_MVSK = (returns_delta * w_opt_MVSK)[1]
        #returns_p_MVSK_static = (returns_delta * w_opt_MVSK_static)[1]
        returns_p_EW = (returns_delta * (ones(length(assets)) * 1/length(assets)))[1]

        realized_returns[i, 1] = returns_p_MV
        realized_returns[i, 2] = returns_p_MV_static
        #realized_returns[i, 3] = returns_p_MVSK
        #realized_returns[i, 4] = returns_p_MVSK_static
        realized_returns[i, 5] = returns_p_EW

        weights[i, :, 1] = w_opt_MV
        #weights[i, :, 2] = w_opt_MVSK

        #Recompute optimal strategy 
        returns_i = returns[1+i - 1 : investment_horizon + i - 1, :]

        portfolio_i = create_portfolio(returns_i, assets);

        ret, risk, w_opt_MV = MV(portfolio_i, λ_MV; method="DCP", w_upper=w_upper)
        
        #results, w_opt_MVSK = MVSK(portfolio_i, λ_MVSK; w_upper=w_upper)

        portfolio_value[i+1, 1] = portfolio_value[i, 1] * (1+returns_p_MV)
        portfolio_value[i+1, 2] = portfolio_value[i, 2] * (1+returns_p_MV_static)
        #portfolio_value[i+1, 3] = portfolio_value[i, 3] * (1+returns_p_MVSK)
        #portfolio_value[i+1, 4] = portfolio_value[i, 4] * (1+returns_p_MVSK_static)
        portfolio_value[i+1, 5] = portfolio_value[i, 5] * (1+returns_p_EW)
        
        start_date = start_date + Month(1)

    end
    
    return realized_returns, portfolio_value, weights, w_opt_MV_static
    
end

### Download data 

In [ ]:
start_date = Date("2000-01-01")
end_date = Date("2022-01-01")

freq = "1M"

#Top-10 in Fortune 500 list from year 2000 
##General Motors is top 1 but only available since 2010
assets = ["WMT", "XOM", "F", "GE", "IBM", "C", "T", "MO", "BA", "BAC", "HPQ", "KR", "AIG", "HD", "PG"] 

df = yf.download(assets, start_date, end_date, progress=false).resample(freq).mean()

df_market = yf.download("^GSPC", start_date, end_date, progress=false).resample(freq).mean()

returns = df["Adj Close"].pct_change().dropna().values

returns_market = df_market["Adj Close"].pct_change().dropna().values;

### Preliminar analisis 

In [ ]:
portfolio = create_portfolio(returns, assets)

RetRisk, weights = MV_efficient_frontier(portfolio, 100)

SR = RetRisk[:, 1] ./ RetRisk[:, 2]

λs = range(0, 1, length=100)

λ_MV = round(λs[argmax(SR)], digits=1)

λs_MVSK = [0.2, 0.2, 0.1, 0.5]

result_MVSK, w_opt_MVSK = @time MVSK(portfolio, λs_MVSK)

cplot = plt.scatter(RetRisk[:, 2], RetRisk[:, 1], c=SR, cmap="jet")
plt.scatter(RetRisk[:, 2][argmax(SR)], RetRisk[:, 1][argmax(SR)], marker="o", facecolor="None", color="k", 
    s=500, lw=3, label="MV")

plt.scatter(result_MVSK[2], result_MVSK[1], marker="*", facecolor="k", color="k", s=500, lw=3, label="MVSK")

cbar = plt.colorbar(cplot)

cbar.set_label("Sharpe Ratio", labelpad=20)

plt.legend();

### Example 

In [ ]:
start_date = Date("2000-01-01")
end_date = Date("2022-01-01")

freq = "1M"
investment_horizon = 12 * 1 #In same units as freq

#Top-10 in Fortune 500 list from year 2000 
##General Motors is top 1 but only available since 2010
assets = ["WMT", "XOM", "F", "GE", "IBM", "C", "T", "MO", "BA", "BAC", "HPQ", "KR", "AIG", "HD", "PG"]

λ_MV = 0.8
λ_MVSK = [0.2, 0.2, 0.1, 0.5]

w_upper = 1.0

realized_returns, portfolio_value, weights, w_opt_MV_static = @time dynamic_optimization(assets, start_date,
    end_date, freq, investment_horizon, λ_MV, λ_MVSK; w_upper=w_upper);

In [ ]:
plt.figure(figsize=(8*2, 6*2))

legend_elements = [L2D([0], [0], lw=3, color="C0", label="MV-D"),
                   L2D([0], [0], lw=3, color="C1", label="MV-S"),
                   L2D([0], [0], lw=3, color="C2", label="MVSK-D"),
                   L2D([0], [0], lw=3, color="C3", label="MVSK-S"),
                   L2D([0], [0], lw=3, color="C4", label="EW"),
                   L2D([0], [0], lw=3, color="k", label="S&P 500")]

dates = start_date  + Year(1) : Month(24) : end_date

market_value = df_market["Adj Close"].values / df_market["Adj Close"].values[1]

plt.subplot(2, 2, 1)

plt.plot(1:size(portfolio_value)[1], portfolio_value[:, 1], color="C0", lw=3, label="MV-D");
plt.plot(1:size(portfolio_value)[1], portfolio_value[:, 2], color="C1", lw=3, label="MV-S")
plt.plot(1:size(portfolio_value)[1], portfolio_value[:, 3], color="C2", lw=3, label="MVSK-D")
plt.plot(1:size(portfolio_value)[1], portfolio_value[:, 4], color="C3", lw=3, label="MVSK-S")
plt.plot(1:size(portfolio_value)[1], portfolio_value[:, 5], color="C4", lw=3, label="EW")

plt.plot(1:size(portfolio_value)[1], market_value, color="k", lw=3)

plt.ylabel("Portfolio value")
plt.xticks(1:24:size(realized_returns)[1], 2000:2:2021, rotation=45)

plt.yscale("log")

plt.legend(handles=legend_elements, loc="upper left", ncol=6, bbox_to_anchor=(-0.05, 1.25))

plt.subplot(2,2,2)

plt.boxplot([realized_returns[:, 1], realized_returns[:, 2], realized_returns[:, 3], realized_returns[:, 4],
        realized_returns[:, 5], returns_market])

plt.xticks([1, 2, 3, 4, 5, 6], ["MV-D", "MV-S", "MVSK-D", "MVSK-S", "EW", "SP500"])
plt.ylabel("Returns")

plt.subplot(2, 2, 3)

max_year_drawdown = zeros((Int(round(Dates.value(end_date - start_date) / (30*12))), 5))
RoMaD = zeros((Int(round(Dates.value(end_date - start_date) / (30*12))), 5))

max_year_drawdown_market = zeros((Int(round(Dates.value(end_date - start_date) / (30*12)))))
RoMaD_market = zeros((Int(round(Dates.value(end_date - start_date) / (30*12)))))

for i in 1 : 22
    
    portfolio_value_i = portfolio_value[12*(i-1) + 1 : 12*i, :]
    market_value_i = market_value[12*(i-1) + 1 : 12*i]
    
    DD, MDD = maximum_drawdown(portfolio_value_i)
 
    DD_market, MDD_market = maximum_drawdown(reshape(market_value_i, (length(market_value_i), 1)))
    
    mean_return = mean(realized_returns[12*(i-1) + 1 : 12*i, :], dims=1) * 100
    
    mean_return_market = mean(returns_market[12*(i-1) + 1 : 12*i]) * 100
    
    max_year_drawdown[i, :] = MDD
    RoMaD[i, :] = - mean_return ./ MDD
            
    max_year_drawdown_market[i, :] = MDD_market
    RoMaD_market[i, :] = - mean_return_market ./ MDD_market
    
end

plt.plot(max_year_drawdown, marker="o", lw=3)
plt.plot(max_year_drawdown_market, marker="o", lw=3, color="k")
        
plt.xticks(0:2:21, 2000:2:2021, rotation=45)
plt.ylabel("Monthly MDD [%]")

plt.subplot(2, 2, 4)

plt.plot(RoMaD, marker="o", lw=3)
plt.plot(RoMaD_market, marker="o", lw=3, color="k")

plt.ylabel("RoMaD")

plt.xticks(0:2:21, 2000:2:2021, rotation=45)

plt.subplots_adjust(hspace=0.25, wspace=0.3)

println("Mean returns: ", ["$(i)%" for i in mean(realized_returns, dims=1).*100])
println("Mean volatility: ", ["$(i)%" for i in sqrt.(var(realized_returns, dims=1)) .* 100])
println("Sharpe ratio: ", ["$(i)%" for i in mean(realized_returns, dims=1) 
            ./ sqrt.(var(realized_returns, dims=1)) ])
println("Max MDD", ["$(i)" for i in minimum(max_year_drawdown, dims=1)])
println("Mean MDD", ["$(i)" for i in mean(max_year_drawdown, dims=1)])
println("Mean RoMaD", ["$(i)" for i in mean(RoMaD, dims=1)])

In [ ]:
##Run each date as an independent job in a cluster parallelizing over 10 cores

#=
start_dates = [Date("2020-01-01")]#, Date("2005-01-01"), Date("2010-01-01"), Date("2015-01-01")]

end_date = Date("2022-01-01")

N = 10

λ_MV = 0.8
λ_MVSK = [0.2, 0.2, 0.1, 0.5]

w_upper = 1.0

@time for start_date in start_dates

    Mean_returns = SharedArray{Float64}(N, 5)
    Mean_volatility = SharedArray{Float64}(N, 5)
    CAGR_t = SharedArray{Float64}(N, 5)
    Max_MDD = SharedArray{Float64}(N, 5)
    Mean_MDD = SharedArray{Float64}(N, 5)
    Mean_RoMaD = SharedArray{Float64}(N, 5)

    #for i in 1 : N 
    @sync @distributed for i in 1 : N
        
        freq = "1M"
        investment_horizon = 12 * i #In same units as freq

        assets = ["WMT", "XOM", "F", "GE", "IBM", "C", "T", "MO", "BA", "BAC", "HPQ", "KR", "AIG", "HD", "PG"]

        realized_returns, portfolio_value, weights, w_opt_MV_static = dynamic_optimization(assets, start_date,
            end_date, freq, investment_horizon, λ_MV, λ_MVSK);

        max_year_drawdown = zeros((22, 5))
        RoMaD = zeros((22, 5))

        for i in 1 : 22

            portfolio_value_i = portfolio_value[12*(i-1) + 1 : 12*i, :]

            DD, MDD = maximum_drawdown(portfolio_value_i)

            mean_return = mean(realized_returns[12*(i-1) + 1 : 12*i, :], dims=1) * 100

            max_year_drawdown[i, :] = MDD
            RoMaD[i, :] = - mean_return ./ MDD

        end

        Mean_returns[i, :] = mean(realized_returns, dims=1)
        Mean_volatility[i, :] = sqrt.(var(realized_returns, dims=1))

        CAGR_MV_D = portfolio_value[:, 1][end]^(1/length(portfolio_value[:, 1])) - 1.0
        CAGR_MV_S = portfolio_value[:, 2][end]^(1/length(portfolio_value[:, 2])) - 1.0
        CAGR_MVSK_D = portfolio_value[:, 3][end]^(1/length(portfolio_value[:, 3])) - 1.0
        CAGR_MVSK_S = portfolio_value[:, 4][end]^(1/length(portfolio_value[:, 4])) - 1.0
        CAGR_EW = portfolio_value[:, 5][end]^(1/length(portfolio_value[:, 5])) - 1.0

        CAGR_t[i, :] = [CAGR_MV_D, CAGR_MV_S, CAGR_MVSK_D, CAGR_MVSK_S, CAGR_EW]

        Max_MDD[i, :] = minimum(max_year_drawdown, dims=1)
        Mean_MDD[i, :] = mean(max_year_drawdown, dims=1)
        
        Mean_RoMaD[i, :] = mean(RoMaD, dims=1)

    end
            
    f1 = open("Dynamic_Optimization_Results/Returns_$(start_date).txt", "w")
    f2 = open("Dynamic_Optimization_Results/Volatility_$(start_date).txt", "w")
    f3 = open("Dynamic_Optimization_Results/CAGR_$(start_date).txt", "w")
    f4 = open("Dynamic_Optimization_Results/Max_MDD_$(start_date).txt", "w")
    f5 = open("Dynamic_Optimization_Results/Mean_MDD_$(start_date).txt", "w")
    f6 = open("Dynamic_Optimization_Results/Mean_RoMaD$(start_date).txt", "w")

    writedlm(f1, Mean_returns)
    writedlm(f2, Mean_volatility)
    writedlm(f3, CAGR_t)
    writedlm(f4, Max_MDD)
    writedlm(f5, Mean_MDD)
    writedlm(f6, Mean_RoMaD)

    close(f1)
    close(f2)
    close(f3)
    close(f4)
    close(f5)
    close(f6)
    
end
=#

##Load Results
start_dates = [Date("2005-01-01"), Date("2010-01-01"), Date("2015-01-01")]

j = 0

plt.figure(figsize=(8*2, 6*2))

legend_elements = [L2D([0], [0], lw=3, color="C0", label="MV-D"),
                   L2D([0], [0], lw=3, color="C1", label="MV-S"),
                   L2D([0], [0], lw=3, color="C2", label="MVSK-D"),
                   L2D([0], [0], lw=3, color="C3", label="MVSK-S"),
                   L2D([0], [0], lw=3, color="C4", label="EW")]

for start_date in start_dates
    
    j += 1
   
    data = readdlm("Dynamic_Optimization_Results/CAGR_$(start_date).txt")
    
    N = size(data)[1]
    
    plt.subplot(2, 2, j)
    
    plt.plot(1:N, data, marker="o", lw=3, ms=12)
    
    plt.ylabel("CAGR")
    plt.xlabel("Lookback period [Years]")
    
    plt.title(start_date)
        
end

plt.subplot(2, 2, 1)
plt.legend(handles=legend_elements, loc="upper left", ncol=5, bbox_to_anchor=(0.2, 1.35))

plt.subplots_adjust(hspace=0.4, wspace=0.3)

With respect to CAGR:

- MVSK-Static always outperforms MV-Static (except for 2015). This is in good agreement with our previous results.

- MV-Dynamic outperforms all the other portfolios, including MVSK-Dynamic. The best loockback period is 1 year, as was already observed in PAPER.

# Yield Curve 

In [ ]:
function clean_df(df)
    
    df["Date"] = pd.to_datetime(df["Date"])
    
    df = df.set_index("Date")
    
    df = df.drop(["20 YR", "30 YR", "Extrapolation Factor",
       "8 WEEKS BANK DISCOUNT", "COUPON EQUIVALENT", "52 WEEKS BANK DISCOUNT",
       "COUPON EQUIVALENT.1","2 Mo"], axis=1)
    
    return df

end

function get_YC_data(years)

    year = years[1]

    url = "https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_yield_curve&field_tdr_date_value=$(year)"

    YC_df = clean_df(pd.read_html(url)[1])

    for year in years[2:end]

        url = "https://home.treasury.gov/resource-center/data-chart-center/interest-rates/TextView?type=daily_treasury_yield_curve&field_tdr_date_value=$(year)"

        if year == years[1]

            YC_df = clean_df(pd.read_html(url)[1])

        else

            YC_df = pd.concat([YC_df, clean_df(pd.read_html(url)[1])])

        end

    end
        
    return YC_df
    
end

In [ ]:
#2011-2022 period
years_train = 2013 : 1 : 2016

YC_df_train = get_YC_data(years_train)

#2017-2020 period
years_test = 2017 : 1 : 2019

YC_df_test = get_YC_data(years_test)

#Long 1990-2022 period
years = 1990: 1 : 2022

YC_df = get_YC_data(years)

#YC_df_avg = YC_df.resample("1Y").mean();

YC_df_train_avg = YC_df_train.resample("1Y").mean()
YC_df_test_avg = YC_df_test.resample("1Y").mean();

In [ ]:
colors = matplotlib.cm.get_cmap("viridis", length(YC_df.columns))

colnames = [item for item in YC_df.columns]

legend_elements = []

for i in 1 : length(YC_df.columns)
   
    append!(legend_elements, [L2D([0], [0], color=colors(i), lw=5, label=colnames[i])])
    
end

append!(legend_elements, [L2D([0], [0], color="b", ls=":", lw=2, label="10Yr-2Yr spread")])

fig, ax = plt.subplot_mosaic("""AA
    BC""", figsize=((8*2, 6*2.2)))

YC_df.plot(ax=ax["A"], cmap="viridis")

#COVID crisis
circle1 = Ellipse((YC_df.index.values[7550], 1.4), 300, 3.5, fill=false, color="red", lw=3, ls="-", zorder=2)

ax["A"].text(YC_df.index.values[6700], 6, "COVID-19 crisis")
ax["A"].annotate("", (YC_df.index.values[7550], 3.1), (YC_df.index.values[7550], 5.8), 
    arrowprops=Dict("arrowstyle"=>"->", "color"=>"k", "linewidth"=>2),)
ax["A"].add_patch(circle1)

#2008 crisis (Great Recession)
circle2 = Ellipse((YC_df.index.values[4550], 2.9), 500, 6, fill=false, color="red", lw=3, zorder=2)

ax["A"].text(YC_df.index.values[3200], 9, "Great Recession")
ax["A"].annotate("", (YC_df.index.values[4400], 6), (YC_df.index.values[4100], 8.8), 
    arrowprops=Dict("arrowstyle"=>"->", "color"=>"k", "linewidth"=>2),)
ax["A"].add_patch(circle2)

ax["A"].set_xlabel("")
ax["A"].set_ylabel("Yield [%]")

ax2 = ax["A"].twinx()

ax2.plot(YC_df["10 Yr"] - YC_df["2 Yr"], color="b", ls=":", alpha=1, label="10Yr-2Yr spread")

ax2.set_ylabel("Spread", color="b")

ax["A"].legend(handles=legend_elements, ncol=6, loc="upper center", bbox_to_anchor=(0.5, 1.3))

for i in 1 : length(YC_df_train_avg)

    YC_df_train_avg.iloc[i].plot(ax=ax["B"], label="$(years_train[i])", lw=3, marker="o")
        
    #ax["B"].legend(ncol=3, loc="upper center", bbox_to_anchor=(0.5, 1.35))
    ax["B"].legend(ncol=1, loc="upper left")
    
end

for i in 1 : length(YC_df_test_avg)

    YC_df_test_avg.iloc[i].plot(ax=ax["C"], label="$(years_test[i])", lw=3, marker="o")
    
    ax["C"].legend(ncol=1, loc="lower right")#, bbox_to_anchor=(0.5, 1.35))
    
end

plt.subplots_adjust(hspace=0.25)

# Auxiliar things 

In [ ]:
returns = df["Adj Close"].pct_change().dropna().values

returns_test_1 = df_test_1["Adj Close"].pct_change().dropna().values
returns_test_2 = df_test_2["Adj Close"].pct_change().dropna().values
returns_test_3 = df_test_3["Adj Close"].pct_change().dropna().values

assets = string.(zeros((1, 10)))

i = 0

for item in df["Adj Close"].columns
   
    i += 1
    
    assets[i] = item
    
end

f = open("returns.txt", "w")

f1 = open("returns_test_1.txt", "w")
f2 = open("returns_test_2.txt", "w")
f3 = open("returns_test_3.txt", "w")

#f2 = open("log_returns.txt", "w")
#f4 = open("log_returns_test.txt", "w")

writedlm(f, assets)
writedlm(f, returns)

writedlm(f1, assets)
writedlm(f1, returns_test_1)

writedlm(f2, assets)
writedlm(f2, returns_test_2)

writedlm(f3, assets)
writedlm(f3, returns_test_3)

close(f)

close(f1)
close(f2)
close(f3)

In [ ]:
λs = []

δ = 0.02

for i in 0.0 : δ : 1.0
    
    for j in 0.0 : δ : 1.0
        
        for k in 0.0 : δ : 1.0
            
            for l in 0.0 : δ : 1.0
                
                if i+j+k+l == 1.0
                   
                    append!(λs, [[i,j,k,l]])
                    
                end
                
            end
            
        end
        
    end
    
end

λs

In [ ]:
λs = hcat(0.0 : 0.1 : 1.0, 1.0 : -0.1 : 0.0, zeros(11), zeros(11))

In [ ]:
RESULTS = zeros((size(λs)[1], 4))

for i in 1 : size(λs)[1]

    println("λ=", λs[i, :])
    
    res, weights = @time MVSK(portfolio_test_1, λs[i, :])
    
    RESULTS[i, :] = res
    
end

In [ ]:
plt.plot(RetRisk_test_1[:, 2], RetRisk_test_1[:, 1], ls="-", lw=5, color="r")

plt.scatter(RESULTS[:, 2], RESULTS[:, 1], s=200)

plt.scatter(results_OOS_MV_opt_1[:,2], results_OOS_MV_opt_1[:, 1], color="g", s=200) 